In [ ]:
!pip install opencv-python img2pdf
import cv2
import os
import numpy as np
import shutil
import img2pdf
from google.colab import files

In [9]:
print("silakan upload video Anda...")
uploaded = files.upload()

video_name = list(uploaded.keys())[0]
nama_file_bersih = os.path.splitext(video_name)[0]

print(f"\n[Info] Video diterima: {video_name}")
print(f"[Info] ID Project: {nama_file_bersih}")

silakan upload video Anda...


Saving pert-11.mp4 to pert-11 (1).mp4

[Info] Video diterima: pert-11 (1).mp4
[Info] ID Project: pert-11 (1)


In [10]:
def extract_unique_frames(video_path, output_dir, threshold=7.0):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    cap = cv2.VideoCapture(video_path)
    prev_frame_gray = None
    saved_count = 0
    frame_count = 0

    print(f"[Proses] Membaca video dengan Threshold {threshold}...")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (9, 9), 0)

        should_save = False

        if prev_frame_gray is None:
            should_save = True
        else:
            diff = cv2.absdiff(prev_frame_gray, gray)
            score = np.mean(diff)

            if score > threshold:
                should_save = True

        if should_save:
            filename = os.path.join(output_dir, f"frame_{frame_count:05d}.jpg")
            cv2.imwrite(filename, frame)
            prev_frame_gray = gray
            saved_count += 1

        frame_count += 1

    cap.release()
    return frame_count, saved_count

In [11]:
OUTPUT_FOLDER = "temp_frames"
THRESHOLD = 7.0

total_f, saved_f = extract_unique_frames(video_name, OUTPUT_FOLDER, THRESHOLD)

print("="*30)
print(f"Total Frame Video : {total_f}")
print(f"Frame Unik Disimpan: {saved_f}")
print("="*30)

[Proses] Membaca video dengan Threshold 7.0...
Total Frame Video : 92144
Frame Unik Disimpan: 18


In [12]:
zip_filename = f"{nama_file_bersih}.zip"
print(f"[Info] Mengompresi gambar ke {zip_filename}...")

shutil.make_archive(nama_file_bersih, 'zip', OUTPUT_FOLDER)
print("Selesai. Siap didownload.")

[Info] Mengompresi gambar ke pert-11 (1).zip...
Selesai. Siap didownload.


In [13]:
pdf_filename = f"{nama_file_bersih}.pdf"
print(f"[Info] Menyusun {saved_f} gambar menjadi PDF: {pdf_filename}...")

imgs = [os.path.join(OUTPUT_FOLDER, f) for f in os.listdir(OUTPUT_FOLDER) if f.endswith(".jpg")]
imgs.sort()

with open(pdf_filename, "wb") as f:
    f.write(img2pdf.convert(imgs))

print("PDF berhasil dibuat.")

[Info] Menyusun 18 gambar menjadi PDF: pert-11 (1).pdf...
PDF berhasil dibuat.
